# Advanced concepts

## Decorators

A Python decorator is a specific change to the Python syntax that allows us to more conveniently alter functions and methods.

In [ ]:
def decorator(f):
    print("Decorator called")
    
    def wrapper():
        print("Wrapper called")
        f()
        
    return wrapper
    
@decorator
def nothing_doer():
    print("Decorated function called")
    
nothing_doer()
print(nothing_doer.__name__)

 It's syntax sugar, equivalent code:

In [ ]:
def decorator(f):
    print("Decorator called")
    
    def wrapper():
        print("Wrapper called")
        f()
        
    return wrapper

def nothing_doer():
    print("Decorated function called")
    
wrapped = decorator(nothing_doer)
wrapped()

Nested decorators:

In [ ]:
def decorator1(f):
    print("Decorator 1 called")
    
    def wrapper():
        print("Decorator 1 wrapper called")
        f()
        
    return wrapper
        
def decorator2(f):
    print("Decorator 2 called")
    
    def wrapper():
        print("Decorator 2 wrapper called")
        f()
        
    return wrapper

@decorator1
@decorator2
def nothing_doer():
    print("Decorated function called")
    
nothing_doer()

Equivalent code:

In [ ]:
def decorator1(f):
    print("Decorator 1 called")
    
    def wrapper():
        print("Decorator 1 wrapper called")
        f()
        
    return wrapper
        
def decorator2(f):
    print("Decorator 2 called")
    
    def wrapper():
        print("Decorator 2 wrapper called")
        f()
        
    return wrapper

def nothing_doer():
    print("Decorated function called")
    
wrapped = decorator1(decorator2(nothing_doer))
wrapped()

Parametrized decorator:

In [ ]:
myvar = "World"

def superdecorator(message, subject):
    def decorator(f):
        def wrapper():
            print("Wrapper called: " + message + " " + subject)
            f()

        return wrapper
    
    return decorator

@superdecorator("Hello", myvar)
def nothing_doer():
    print("Decorated function called")
    
nothing_doer()

Class property decorator:

In [ ]:
class Celsius:
    def __init__(self, temperature = 0):
        self._temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    @property
    def temperature(self):
        print("Getting value")
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value
        
c = Celsius()
c.temperature = 36.6
c.temperature

How it works:

[Documentation on property()](https://docs.python.org/3/library/functions.html#property)

First, equivalent code:

In [ ]:
class MyClass:
    def getter(self):
        print("Getter called")

    def setter(self, arg):
        print("Setter called: {}".format(arg))

    x = property(getter, setter)

m = MyClass()
m.x
m.x = 12

Let's write our own property():

In [ ]:
class MyProperty:
    def __init__(self, f):
        self._getter = f
        
    def __get__(self, instance, klass):
        return self._getter(instance)
       
    def __set__(self, instance, value):
        return self._setter(instance, value)
        
    def setter(self, f):
        self._setter = f

def myproperty(f):
    return MyProperty(f)

class MyClass:
    def __init__(self, x):
        self._x = x
    
    @myproperty
    def x(self):
        print("Getter called: {}".format(self._x))
        return self._x

    @x.setter
    def setter(self, arg):
        print("Setter called: {}".format(arg))
        self._x = arg

m = MyClass(10)
print(m.x)

m.x = 12
print(m.x)

### Memoize pattern

Imagine factorial() contains some expensive computations:

In [ ]:
def memoize(f):
    cache = {}
    def wrapper(x):
        if x in cache:
            print("Cache hit: {}".format(x))
            return cache[x]
        
        print("wrapper({})".format(x))
        
        r = f(x)
        cache[x] = r
        
        return r
        
    return wrapper

@memoize
def factorial(x):
    return 1 if x == 1 else x * factorial(x - 1)

print(factorial(3))
print(factorial(4))

### Module 'functools'

[Docs](https://docs.python.org/3/library/functools.html)

**@functools.lru_cache()**: Don't reinvent the wheel + some goodies

In [ ]:
import functools

@functools.lru_cache(maxsize=6)
def factorial(x):
    return 1 if x == 1 else x * factorial(x - 1)

for n in range(1, 6):
    factorial(n)

factorial.cache_info()

**functools.partial()**: [Stackoverflow: What is currying](https://stackoverflow.com/questions/36314/what-is-currying)

In [ ]:
def manyargs(a, b, c):
    print("a={}, b={}, c={}".format(a, b, c))
    
p1 = functools.partial(manyargs, 1)
p2 = functools.partial(p1, 2)

p2(3)

In [ ]:
p1 = functools.partial(manyargs, c=1)
p2 = functools.partial(p1, b=2)

p2(a=3)

**@functools.wraps()**: fixes wrapped function metadata

In [ ]:
def decorator(f):
    @functools.wraps(f)
    def wrapper():
        print("Wrapper called")        
        f()
        
    return wrapper
    
@decorator
def nothing_doer():
    print("Decorated function called")
    
nothing_doer()

print(nothing_doer.__name__)

## Iterators

In [ ]:
it = iter([1, 2, 3])
it

In [ ]:
print(next(it))
print(next(it))
print(next(it))

In [ ]:
next(it)

Iterator object and protocol

In [ ]:
import random

class MyIterator:
    def __init__(self, iterations, stop):
        self._iterations = iterations
        self._stop = stop

    def __iter__(self):
        """Or get TypeError: 'MyIterator' object is not iterable"""
        return self
    
    def __next__(self):
        if self._iterations < 1:
            raise StopIteration
        
        self._iterations -= 1
        
        return random.randrange(self._stop)
    
myiter = MyIterator(5, 10)
list(x for x in myiter)

## Generators

A generator is a function that produces a sequence of results instead of a single value.

In [ ]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1
    
for x in countdown(5):
    print(x)

Btw, remember list comprehensions?

In [ ]:
[x for x in range(1, 10)]

Here, the generator equvalent:

In [ ]:
(x for x in range(1, 10))

So this expression has low memory footprint:

In [ ]:
sum(x for x in range(1, 10))

Generators do not act like normal functions when called:

In [ ]:
def countdown(n):
    print("countdown({}) called".format(n))
    
    if n == 101:
        return "no-no"
    
    while n > 0:
        yield n
        n -= 1
        
    print("end")

cd = countdown(5)

print(cd)
        
for x in cd:
    print(x)

for x in countdown(101):
    print(x)

Generators conform iteration protocol:

In [ ]:
cd = countdown(2)

print(next(cd))
print(next(cd))
print(next(cd))

Generator pipeline:

In [ ]:
import os

def ls(d):
    return [(yield x) for x in os.listdir(d)]

def files(f):
    return [(yield x) for x in f if os.path.isfile(x)]

def workshops(f):
    return [(yield x) for x in f if x.startswith("Workshop")]

list(workshops(files(ls("."))))

### Module 'itertools'

[Docs](https://docs.python.org/3/library/itertools.html)

In [ ]:
import itertools

**itertools.count(start, \[step\])**: generate numbers endlessly

In [ ]:
it = itertools.count(5)

print(next(it))
print(next(it))
print(next(it))

**itertools.cycle(iterable)**: cycle thru values endlessly

In [ ]:
it = itertools.cycle([0, 1])

print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))

**itertools.repeat(elem \[,n\])**: endlessly or up to n times

In [ ]:
list(itertools.repeat('A', 10))

**itertools.zip_longest(*iterables, fillvalue=None)**: zip() that stops on longer list

In [ ]:
list(itertools.zip_longest('ABCDE', [0, 1], fillvalue=-1))

**itertools.chain(*iterables)**: Make an iterator that returns elements from the first iterable until it is exhausted, then proceeds to the next iterable, until all of the iterables are exhausted.

In [ ]:
list(itertools.chain('ABCD', [0, 1]))

* **itertools.combinations(iterable, r)**: Return r length subsequences of elements from the input iterable.
* **itertools.combinations_with_replacement(iterable, r)**: Return r length subsequences of elements from the input iterable allowing individual elements to be repeated more than once.

In [ ]:
list(itertools.combinations('ABC', 2))

In [ ]:
list(itertools.combinations_with_replacement('ABC', 2))

### Module 'collections'

[Docs](https://docs.python.org/3/library/collections.html)

In [ ]:
import collections

**collections.defaultdict(default_factory)**: When each key is encountered for the first time, it is not already in the mapping; so an entry is automatically created using the default_factory function

In [ ]:
d = collections.defaultdict(list)

d['students'].append('Vasya')
dict(d)

**collections.OrderedDict([items])**: An OrderedDict is a dict that remembers the order that keys were first inserted. If a new entry overwrites an existing entry, the original insertion position is left unchanged. Deleting an entry and reinserting it will move it to the end.

In [ ]:
d = collections.OrderedDict()
d['a'] = 1
d['b'] = 2
d['c'] = 3

d

**collections.namedtuple(typename, field_names, ...)**: Named tuples assign meaning to each position in a tuple and allow for more readable, self-documenting code. They can be used wherever regular tuples are used, and they add the ability to access fields by name instead of position index.

In [ ]:
Student = collections.namedtuple('Student', 'name, surname, age')

john = Student('John', 'Smith', '21')

tuple(john)

In [ ]:
john.age

In [ ]:
str(john)

**collections.deque([iterable[, maxlen]])**: Deques are a generalization of stacks and queues (the name is pronounced “deck” and is short for “double-ended queue”)

In [ ]:
d = collections.deque()

d.append('a')
d.append('b')
d.append('c')

d

In [ ]:
print(d.popleft())
d

In [ ]:
d.appendleft('Ghost!')
d

**collections.Counter([iterable-or-mapping])**: A Counter is a dict subclass for counting hashable objects.

In [ ]:
counts = collections.Counter('cowabunga')
counts

In [ ]:
counts.most_common(1)

## Context managers

Simplifies try/finally. Common usage:

In [ ]:
with open('README.md') as f:
    contents = f.read()

f.closed

### Context manager protocol

In [ ]:
class MyContextManager:
    def __enter__(self):
        print("Entered")
        
    def __exit__(self, exc_type, exc_value, traceback):
        print("Exited with {}: {}".format(exc_type, exc_value))
        return True
        
with MyContextManager() as mycontext:
    print("I'm ok")

In [ ]:
with MyContextManager() as mycontext:
    raise Exception("Mayday, mayday!")

### Module 'contextlib'

[Docs](https://docs.python.org/3/library/contextlib.html)

**@contextlib.contextmanager**: This function is a decorator that can be used to define a factory function for with statement context managers, without needing to create a class or separate ```__enter__()``` and ```__exit__()``` methods.

In [ ]:
import contextlib

@contextlib.contextmanager
def tag(name):
    print("<%s>" % name)
    yield name
    print("</%s>" % name)

with tag("strong") as tag1:
    with tag("i") as tag2:    
        print("A title: {}/{}".format(tag1, tag2))

In [ ]:
@contextlib.contextmanager
def tag(name):
    print("<%s>" % name)
    try:
        yield
    except:
        print("Got exception")
        
    print("</%s>" % name)

with tag("strong"):
    raise Exception('Test')

## Modules

* The ```__init__.py``` files are required to make Python treat the directories as containing packages
* ```__init__.py``` can just be an empty file, but it can also execute initialization code for the package or set the ```__all__``` variable

Layout:

```
├── mymodule
|   ├── mysubmodule
│   │   ├── __init__.py
|   ├── my.py
|   ├── __init__.py
```

Example:

```
from mymodule.my import myfun
from mymodule import submodule

myfun()
submodule.otherfun()
```

Module aliasing:
    
```
import numpy as np

np.zeros((3, 2))
```

Relative imports:
```
from . import echo
from .. import formats
from ..filters import equalizer
```

Import several packages:
```
import os, sys
```

The import statement uses the following convention: if a package’s ```__init__.py``` code defines a list named ```__all__```, it is taken to be the list of module names that should be imported when from package import * is encountered.

If ```mymodule/mysubmodule/__init__.py``` contains
```
__all__ = ["myfun", "otherfun"]
```

This line:
```
from mymodule.mysubmodule import *
```

will import ```myfun``` and ```otherfun``` only. Wildcard imports are **bad practice**.

## Packages and environments

It's the real pain, some terminology first:

* **PyPI** is the default Package Index for the Python community: [pypi.org](https://pypi.org/)
* **Egg**: A Built Distribution format introduced by **setuptools**, which is being replaced by **Wheel**
* **Wheel**: A Built Distribution format introduced by PEP 427, which is intended to replace the **Egg** format. Wheel is currently supported by **pip**
* **sdist**: A distribution format that provides metadata and the essential source files needed for installing by a tool like pip
* **setup.py**: The project specification file for **distutils** and **setuptools**
* **requirements.txt**: A file containing a list of requirements (dependencies) that can be installed using pip.





### Packaging tools

* **Distutils** is still the standard tool for packaging in Python. It is included in the standard library (Python 2 and Python 3.0 to 3.6). It is useful for simple Python distributions, but lacks features. It introduces the ```distutils``` Python package that can be imported in your setup.py script.
* **Setuptools** was developed to overcome Distutils' limitations, and is not included in the standard library. It introduced a command-line utility called ```easy_install```. It also introduced the ```setuptools``` Python package that can be imported in your setup.py script, and the pkg_resources Python package that can be imported in your code to locate data files installed with a distribution. One of its gotchas is that it monkey-patches the ```distutils``` Python package. It should work well with ```pip```. It sees regular releases.
* **Abandoned**: Distribute, Distutils2
* **Alpha**: Distlib
* **Alternatives**: Bento, Enscons

[Stackoverflow](https://stackoverflow.com/questions/6344076/differences-between-distribute-distutils-setuptools-and-distutils2)

### Package managers

* **PyPi/setuptools**:
    - ```easy_install```: 2004, install from Eggs, no uninstall packages, no Wheel support
    - ```pip```: 2008, no install from Eggs, install from sdist or from Wheel recently, requirement files 
* **Anaconda**: ```conda```
* **Spack**: ```spack```

### Virtual environments

* **Virtualenv**
* **Pyenv**: forked from rbenv and ruby-build, and modified for Python.
* **Anaconda**: package index + package manager + environment manager

Alternatives:
* docker
* vagrant

[Stackoverflow](https://stackoverflow.com/questions/38217545/the-different-between-pyenv-virtualenv-anaconda-in-python)

## Anatomy of a package

```
├── foo
│   ├── a_module.py
│   ├── __init__.py
├── README
└── setup.py
```

setup.py:

```
from setuptools import setup

setup(
   name='foo',
   version='1.0',
   description='A useful module',
   author='Man Foo',
   author_email='foomail@foo.com',
   packages=['foo'],
   install_requires=['bar>=1', 'greek>=1,<2']
)
```

To build & install package:

```python setup.py build && python setup.py install```

## Typical application

```
├── project
│   ├── submodule
│   │   ├── __init__.py
│   │   ├── my.py
│   ├── app.py
│   ├── __init__.py
└── requirements.txt
```

requirements.txt:

```
bar==0.12.2
greek>=1
-e git+git://github.com/path/to/repo@releases/3.7.1#egg=charlie
```

To install application dependencies:

```pip install -r requirements.txt```

## PIP guide

[PIP docs](https://pip.pypa.io/en/stable/)

* ```pip install <package>```
* ```pip uninstall <package>```
* Save from env: ```pip freeze > requirements.txt```
* List outdated: ```pip list --outdated --format=freeze```
* Upgrade all: ```pip freeze --local | grep -v '^\-e' | cut -d = -f 1  | xargs -n1 pip install -U```

## Anaconda

* Environment manager
* Binary packages (for any arch)
* Community packages [Conda-Forge](https://conda-forge.org/)
* Comes with scientific packages (or use miniconda)
* Anaconda Navigator
* Supports Python, R and Julia

### environment.yml

```
name: myapp

channels:
  - conda-forge

dependencies:
  - python=3.6
  - foo
  - bar
  - pip:
    - pip-foo
    - pip-bar
    - git+git://github.com/path/to/repo@releases/3.7.1#egg=charlie
```

### Conda guide

[Conda docs](https://conda.io/docs/user-guide/)

* To create an environment with a specific version of Python and package foo:
```conda create -n myenv python=3.6 foo```
* Install package bar under myenv:
```conda install -n myenv bar```
* Create the environment from the environment.yml file:
```conda env create -f environment.yml```
* List packages installed under myenv:
```conda list -n myenv```
* List local environments:
```conda env list```
* Activate environment myenv:
```source activate myenv```

Once the environment is activated, you can skip specifying -n myenv. Pip will install packages under current env.